In [1]:
# importamos librerías
import pandas as pd

# **Análsis inicial**

In [2]:
# cargamos el archivo con los datos
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet")
df_raw.sample(15)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
11706,db7f5969-6755-4e48-892c-ff744d3133d4,3e23aee2-83af-4f77-961c-271b415ae9c4,Natalio,Naranjo,natalio.naranjo@example.com,False,2025-02-11,2025-03-01,2025-03-02,35,94.59,Hotel Encanto Real,3.0,Madrid
1836,a2ded5ba-1146-402c-b83e-d8fd7b35dab6,58deb094-3dcd-48e6-a092-2d217da02f3e,Jose Ramón,Escamilla,jose ramón.escamilla@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
10386,dc0082c7-fd37-46db-9276-611a8064ea47,0bddf7dc-473e-4ac8-a04b-6a3c005e9bd0,Octavio,Escudero,octavio.escudero@example.com,False,2025-02-12,2025-03-01,2025-03-02,27,308.64,Hotel Camino del Sol,2.0,Madrid
11854,f77ed93b-bfdd-481a-994b-877ce54ee157,c8d5d147-c22d-4f6a-a5b7-0af9508b76bd,Yago,Solís,yago.solís@example.com,False,2025-02-11,2025-03-01,2025-03-02,29,181.78,Hotel Puerta del Cielo,2.0,Madrid
8024,0c3ca00a-d6fd-46a3-9820-672a84aa5d0e,27d32987-acdc-4457-9cce-593462ac2178,Ovidio,Amigó,ovidio.amigó@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,174.09,Gran Hotel Madrid,2.0,Madrid
1828,01f8dd25-f627-4cec-a6e8-ae5fce39e63a,6e5b511c-d84f-4783-a395-8ff014ae864b,Feliciana,Barrera,feliciana.barrera@example.com,False,2025-02-05,2025-03-01,2025-03-02,49,151.02,Hotel Sol y Luna,2.0,Madrid
5188,67b0b598-073c-4fd2-ad38-dabad0f9673f,a94228a6-97a4-4ad6-a55b-3bfb12f6aff6,Renato,Rocha,renato.rocha@example.com,False,2025-02-09,2025-03-01,2025-03-02,36,372.70,Hotel Rincón Sereno,3.0,Madrid
5001,5e59a285-41b4-48f2-ba5c-8958e15d99f7,1c9e2869-6562-4cd6-b1b2-9e0da2c59df2,Maximino,Barberá,maximino.barberá@example.com,False,2025-02-02,2025-03-01,2025-03-02,40,337.73,Gran Hotel Madrid,1.0,Madrid
7771,495a14ff-1284-438a-bf15-6a3fc7a6b9ec,0e6b6e43-9eff-46ff-b639-1f317e652889,Horacio,Gibert,horacio.gibert@example.com,False,2025-02-01,2025-03-01,2025-03-02,15,299.09,Hotel Encanto Real,3.0,Madrid
9495,9ac63f57-e5b1-4b39-8a39-2e278ec78c19,0f5bff7d-a3d0-4459-92e0-862eb92d2e85,Andrea,Castells,andrea.castells@example.com,True,,2025-03-01,2025-03-02,181,NaN,,NaN,


In [3]:
# info del dataframe con los datos
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


In [4]:
# Hacemos copia del dataframe para trabajar
df = df_raw.copy()

# Conteo hoteles competencia

In [5]:
# comprobamos cuántos hoteles son competencia
competencia = df[df["competencia"]==True]
competencia["id_hotel"].nunique()

10

In [6]:
# info del dataframe de hoteles de la competencia
competencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5172 entries, 0 to 14998
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       5172 non-null   object 
 1   id_cliente       5172 non-null   object 
 2   nombre           5172 non-null   object 
 3   apellido         5172 non-null   object 
 4   mail             5172 non-null   object 
 5   competencia      5172 non-null   bool   
 6   fecha_reserva    5172 non-null   object 
 7   inicio_estancia  5149 non-null   object 
 8   final_estancia   5149 non-null   object 
 9   id_hotel         5172 non-null   int64  
 10  precio_noche     0 non-null      float64
 11  nombre_hotel     5172 non-null   object 
 12  estrellas        0 non-null      float64
 13  ciudad           5172 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 570.7+ KB


In [7]:
competencia["nombre_hotel"].unique()

array([''], dtype=object)

Hay 10 hoteles de la competencia sin datos de precio_noche, nombre_hotel y estrellas. Vemos que el valor único de la columna nombre_hotel es " ", lo que implica que están todos vacíos. Tendremos que extraer la información faltante.

# Cambio columnas con fechas a datetime

In [8]:
# Las columnas de fecha_reserva, inicio_estancia y final_estancia deberían estar en formato fecha. Hacemos una función para transformarlas
def trans_fecha(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")
    return dataframe

In [9]:
df = trans_fecha(["fecha_reserva", "inicio_estancia", "final_estancia"], df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

# Fechas estancia

In [10]:
# miramos valores únicos de fechas de inicio y final de estancia porque las necesitaremos para buscar eventos en la API
inicio = df["inicio_estancia"].unique()
inicio

<DatetimeArray>
['2025-03-01 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [11]:
final = df["final_estancia"].unique()
final

<DatetimeArray>
['2025-03-02 00:00:00', 'NaT']
Length: 2, dtype: datetime64[ns]

In [12]:
final[0]

Timestamp('2025-03-02 00:00:00')

Como las fechas son las mismas, podríamos rellenar toda la columna con el mismo valor.

In [13]:
# Rellenamos las columnas del df con las fechas.
df["inicio_estancia"] = inicio[0]
df["final_estancia"] = final[0]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15098 non-null  datetime64[ns]
 8   final_estancia   15098 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

In [15]:
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
5383,d8a45b91-bfb0-4e6b-bf85-af2c00e17c8e,ddd29e93-cf3c-450d-a562-24c94b5b60aa,Santos,Barreda,santos.barreda@example.com,False,2025-02-10,2025-03-01,2025-03-02,19,422.67,Hotel Rincón Sereno,3.0,Madrid
3186,9bea6761-865b-4915-bda8-f9e7601f9755,f7fca634-632a-4668-a15b-bfd76a483e91,Maricruz,Olivé,maricruz.olivé@example.com,False,2025-02-07,2025-03-01,2025-03-02,49,182.71,Palacio del Sol,1.0,Madrid
14101,4813770b-ebb4-4c07-ae71-93d67734eddb,35335beb-d8e3-435d-a9ff-40fbb6ec2f9c,Sandra,Goñi,sandra.goñi@example.com,False,2025-02-01,2025-03-01,2025-03-02,3,93.63,Hotel Sol y Luna,5.0,Madrid
309,760b9113-56ad-4c5f-bb47-74980488d028,b4998190-bade-4504-959e-bbcd40458187,Ciro,Gomila,ciro.gomila@example.com,False,2025-02-12,2025-03-01,2025-03-02,40,149.69,Gran Hotel Madrid,5.0,Madrid
9839,7e95fa9c-43d9-410c-94e9-d4e2a4ff8a41,4f54f5aa-9b68-4370-ad6b-2f7a1b80f868,Anunciación,Belda,anunciación.belda@example.com,False,2025-02-02,2025-03-01,2025-03-02,28,481.96,Hotel Vista Alegre,3.0,Madrid


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15098 non-null  datetime64[ns]
 8   final_estancia   15098 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

# Ciudades

In [17]:
ciudades = df["ciudad"].unique()
ciudades

array(['', 'Madrid'], dtype=object)

In [18]:
# Rellenamos toda la columna con Madrid.
df["ciudad"] = "Madrid"
df.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
7970,41c3c33c-57e2-4d80-8bf1-142e1a780b4c,a9c28149-6965-41f2-9e6a-b7e9c5b07ac2,Sancho,Hierro,sancho.hierro@example.com,False,2025-02-03,2025-03-01,2025-03-02,6,148.31,Hotel Vista Alegre,4.0,Madrid
10222,8bbf1e48-53b6-4736-a582-3acafe7fe6d1,6cccc821-504e-4233-8fe8-7d9fc21ab5d3,Paz,Rocamora,paz.rocamora@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,Madrid
1480,cf0e17f7-197f-4c27-baaf-5e44713bcd72,d1d0e2af-3a81-423f-bc62-ba4a193779b4,Apolinar,Romero,apolinar.romero@example.com,False,2025-02-04,2025-03-01,2025-03-02,49,440.20,Hotel Rincón Sereno,3.0,Madrid
2861,7c77649c-39f1-4c85-9d17-e543879c3b01,3de3ecdf-adfc-425e-8c4d-afac9b29198f,Lucho,Cobo,lucho.cobo@example.com,True,NaT,2025-03-01,2025-03-02,117,NaN,,NaN,Madrid
12280,6494ad69-30c0-4d20-b9a9-164f296b3021,e55382b8-7b64-4636-b8e6-45c5d1f2af59,Mauricio,Calderón,mauricio.calderón@example.com,False,2025-02-01,2025-03-01,2025-03-02,33,331.94,Hotel Torre Dorada,4.0,Madrid


# Duplicados

In [19]:
duplicados = df.duplicated()
duplicados.sum()

np.int64(98)

In [20]:
# Hay 98 filas totalmente duplicadas, las eliminamos
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  15000 non-null  datetime64[ns]
 8   final_estancia   15000 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           15000 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

# **Transformación**

De cara a cargar los datos en la base de datos vamos a generar unas tablas a partir del df.

In [21]:
# Podemos tener id_hotel duplicados porque más de un cliente puede reservar en el mismo hotel y podemos tener id_cliente duplicados porque el mismo cliente puede haber hecho varias reservas
# lo que no podemos tener repetido es el id_reserva. Lo comprobamos
reservas = df["id_reserva"].duplicated().sum()
reservas

np.int64(0)

## Hoteles

In [22]:
# Cantidad de id_hotel
df["id_hotel"].nunique()

29

In [23]:
# Cantidad nombres hotel
df["nombre_hotel"].nunique()

20

Ya hemos visto que nos faltan datos de los hoteles de la competencia, lo que explica la discrepancia en la cantidad de valores únicos. Vamos a generar dos tablas de hoteles, una con los hoteles propios y otra con los de la competencia.

In [24]:
df_competencia = df[["id_hotel", "nombre_hotel", "estrellas", "ciudad"]][df["competencia"]==True]
df_competencia.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad
746,131,,NaN,Madrid
14620,181,,NaN,Madrid
6548,117,,NaN,Madrid
7949,117,,NaN,Madrid
3477,114,,NaN,Madrid


In [ ]:
df_propios = df[["id_hotel",  , "estrellas", "ciudad"]][df["competencia"]==False]
df_propios.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad
9894,14,Hotel Mirador Real,5.0,Madrid
5854,36,Hotel Las Estrellas,1.0,Madrid
8200,19,Hotel Jardines del Rey,4.0,Madrid
13449,33,Hotel Torre Dorada,2.0,Madrid
9565,43,Hotel Puerta del Cielo,3.0,Madrid


Al sacar una muestra vemos que hay dos nombres de hotel con id_hotel distintos. Vamos a asociar un único id_hotel por cada nombre_hotel

In [26]:
nombres=df_propios["nombre_hotel"].unique()
nombres

array(['Hotel Monte Verde', 'Hotel Brisas del Mar',
       'Hotel Camino del Sol', 'Hotel Puerta del Cielo',
       'Hotel Encanto Real', 'Palacio del Sol', 'Hotel Jardines del Rey',
       'Hotel Las Estrellas', 'Gran Hotel Madrid', 'Hotel Torre Dorada',
       'Hotel Palacio Imperial', 'Hotel Luz de Madrid',
       'Hotel Los Almendros', 'Hotel Sol y Luna', 'Hotel Mirador Real',
       'Hotel Rincón Sereno', 'Hotel Vista Alegre', 'Hotel Costa Azul',
       'Hotel Maravilla Real'], dtype=object)

In [27]:
nombrehotel=[]
idhotel=[]
for nombre in enumerate(nombres, start=1):
    nombrehotel.append(nombre[1])
    idhotel.append(f"00{nombre[0]}")
hoteles_propios = dict(zip(nombrehotel, idhotel))
hoteles_propios

{'Hotel Monte Verde': '001',
 'Hotel Brisas del Mar': '002',
 'Hotel Camino del Sol': '003',
 'Hotel Puerta del Cielo': '004',
 'Hotel Encanto Real': '005',
 'Palacio del Sol': '006',
 'Hotel Jardines del Rey': '007',
 'Hotel Las Estrellas': '008',
 'Gran Hotel Madrid': '009',
 'Hotel Torre Dorada': '0010',
 'Hotel Palacio Imperial': '0011',
 'Hotel Luz de Madrid': '0012',
 'Hotel Los Almendros': '0013',
 'Hotel Sol y Luna': '0014',
 'Hotel Mirador Real': '0015',
 'Hotel Rincón Sereno': '0016',
 'Hotel Vista Alegre': '0017',
 'Hotel Costa Azul': '0018',
 'Hotel Maravilla Real': '0019'}

In [29]:
df_propios["id_hotel"] = df_propios["nombre_hotel"].map(hoteles_propios)
df_propios.sample(5)

,id_hotel,nombre_hotel,estrellas,ciudad
6581,002,Hotel Brisas del Mar,4.0,Madrid
596,0013,Hotel Los Almendros,5.0,Madrid
11056,008,Hotel Las Estrellas,3.0,Madrid
12793,0017,Hotel Vista Alegre,1.0,Madrid
4810,009,Gran Hotel Madrid,4.0,Madrid


Una vez que tenemos los id_hotel correctamente asignados, debemos corregir la columna estrellas, que tiene valores distintos para el mismo nombre de hotel

In [30]:
df_estrellas = df_propios.groupby("nombre_hotel")["estrellas"].mean()
df_estrellas

nombre_hotel
Gran Hotel Madrid         3.067568
Hotel Brisas del Mar      3.088015
Hotel Camino del Sol      3.067568
Hotel Costa Azul          3.118447
Hotel Encanto Real        3.031311
Hotel Jardines del Rey    2.936255
Hotel Las Estrellas       2.906615
Hotel Los Almendros       3.011257
Hotel Luz de Madrid       3.052239
Hotel Maravilla Real      2.978858
Hotel Mirador Real        2.977486
Hotel Monte Verde         3.103636
Hotel Palacio Imperial    3.006479
Hotel Puerta del Cielo    3.028517
Hotel Rincón Sereno       2.998047
Hotel Sol y Luna          3.008977
Hotel Torre Dorada        2.919922
Hotel Vista Alegre        2.941292
Palacio del Sol           2.992157
Name: estrellas, dtype: float64